In [6]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import csv

In [7]:
# 웹페이지 접속하기
driver = webdriver.Chrome()
driver.get("https://www.kcsc.re.kr/")
time.sleep(3)

In [10]:
# 웹페이지에 있는 검색창에 텍스트 입력하고 Enter
elements_1 = driver.find_elements(by=By.NAME, value="searchTxt")
if elements_1:
    search_box = elements_1[0]
    search_box.send_keys('KCS 21 30 00')
    search_box.send_keys(Keys.ENTER)
else:
    print("요소를 찾을 수 없습니다.")

#이까지는 성공했는데.. 요소에 있는 txt를 출력해서 검색어랑 비교 후 일치하게 되면 특정 버튼을 클릭하게 하는 코드를 못 짜겠음

In [11]:
# 새로 들어간 페이지에서 '내용보기' 버튼 클릭하기
button_xpath = '//*[@id="scope-search"]/div[1]/div[2]/div[3]/table/tbody/tr/td[4]/a'  # 버튼의 XPath 입력
try:
    button_element = driver.find_element(By.XPATH, button_xpath)
    button_element.click()
except:
    print("버튼을 찾을 수 없거나 클릭할 수 없습니다.")
    
# 이렇게 짜게되면 키워드로 검색된 결과가 무조건 맞다고 생각하고 내용보기 버튼을 클릭하는 것(오류가 좀 있음)

In [5]:
# 새롭게 들어간 페이지를 기본 페이지로 하고 URL 가져오기
all_window_handles = driver.window_handles
main_window_handle = driver.current_window_handle

for handle in all_window_handles:
    if handle != main_window_handle:
        driver.switch_to.window(handle)
        break

new_url = driver.current_url
print(new_url)

https://www.kcsc.re.kr/StandardCode/Viewer/40448/KCS%2041%2033%2001


In [293]:
# 가져온 URL에서 Div class 'code-content' 안에 있는 내용 가져오기

driver.get(new_url)

time.sleep(10)

div_element = driver.find_element(By.CLASS_NAME, 'code-content')

# div 요소 내의 텍스트 추출
div_paragraphs = div_element.find_elements(By.TAG_NAME, 'p')

# div 요소 내의 표 데이터 추출
div_table = div_element.find_element(By.TAG_NAME, 'table')

# BeautifulSoup을 사용하여 표 내부 텍스트 추출
table_html = div_table.get_attribute('outerHTML')
table_soup = BeautifulSoup(table_html, 'html.parser')

table_data = []

if table_soup:
    rows = table_soup.find_all('tr')
    
    for row in rows:
        row_data = []
        cells = row.find_all(['th', 'td'])
        
        for cell in cells:
            cell_text = cell.get_text()
            row_data.append(cell_text)
        
        # 현재 행의 데이터를 표 데이터에 추가
        table_data.append(row_data)

# 텍스트 데이터와 표 데이터를 통합하여 CSV 파일로 저장
csv_file_name = 'C:/Users/User/Desktop/csv/output.csv'

with open(csv_file_name, 'w', newline='', encoding='utf-8') as csv_file:
    csv_writer = csv.writer(csv_file)
    
    # 각 p 태그의 텍스트 추가
    csv_writer.writerow(['Text'])
    for paragraph in div_paragraphs:
        csv_writer.writerow([paragraph.text])
    
    # 빈 줄 추가
    csv_writer.writerow([])
    
    # 표 데이터 추가
    for row_data in table_data:
        csv_writer.writerow(row_data)

# 웹 드라이버 종료
driver.quit()

print(f"텍스트 데이터와 표 데이터를 {csv_file_name} 파일로 저장하였습니다.")

텍스트 데이터와 표 데이터를 C:/Users/User/Desktop/csv/output.csv 파일로 저장하였습니다.
